In [ ]:
import pandas as pd

In [ ]:
from skmultiflow.data.sea_generator import SEAGenerator

stream = SEAGenerator(classification_function = 2, 
                      random_state = 112, 
                      balance_classes = False, 
                      noise_percentage = 0.28)


In [ ]:
X, y = stream.next_sample(1000)

In [ ]:
df = pd.DataFrame(X, columns=['X1','X2','X3'])
df['target'] = y
df.head()

In [ ]:
from skmultiflow.meta import AdaptiveRandomForestRegressor

model = AdaptiveRandomForestRegressor()

model.fit(X, y)


In [ ]:
from skmultiflow.drift_detection.adwin import ADWIN

change_detector = ADWIN()

while stream.has_more_samples():
    X_i, y_i = stream.next_sample()

    pred = model.predict(X_i)

    err = int(pred != y_i)
    change_detector.add_element(err)

    if change_detector.detected_change():
        print('Detected change')


In [ ]:
from datasetsforecast.m3 import M3

dataset, *_ = M3.load('./data', 'Quarterly')

series = dataset.query(f'unique_id=="Q1"')

In [ ]:
from plotnine import *

plot = \
    ggplot(series.reset_index()) + \
    aes(x='index', y='y', group=1) + \
    theme_538(base_family='Palatino', base_size=12) + \
    theme(plot_margin=.125,
          axis_text=element_text(size=12),
          panel_background=element_rect(fill='white'),
          plot_background=element_rect(fill='white'),
          legend_box_background=element_rect(fill='white'),
          strip_background=element_rect(fill='white'),
          legend_background=element_rect(fill='white'),
          legend_title=element_blank(),
          axis_text_x=element_blank(),
          legend_position=None) + \
    geom_line(color='black', size=1) + labs(x='', y='')

plot + theme(figure_size=(13,6))